# Exercice 1
*Les questions sont indépendantes. Dans cette exercice, on n'utilisera pas le module `List`*
1. Ecrire une fonction `exists : ('a -> bool) -> 'a list -> bool` qui prend en argument une fonction `f` et une liste `lst` et qui renvoie `true` si la liste comporte au moins un élément `a` tel que `f a` vaut `true`, `false` sinon.

In [1]:
let rec exists f lst =
  match lst with
  | [] -> false
  | t::q -> f t || exists f q
;;

val exists : ('a -> bool) -> 'a list -> bool = <fun>


2. Ecrire une fonction `for_all : ('a -> bool) -> 'a list -> bool` qui prend en argument une fonction `f` et une liste `lst` et qui renvoie `true` si `f` vaut `true` pour tous les éléments de la liste, `false` sinon.

In [2]:
let rec for_all f lst =
  match lst with
  | [] -> true
  | t::q -> f t && for_all f q
;;

val for_all : ('a -> bool) -> 'a list -> bool = <fun>


3. Le type polymorphe `'a option`, présent dans la bibliothèque standard OCaml, est défini par : 
    ```Ocaml
    type 'a option = 
      | None
      | Some of 'a
      ;;
      ```
     Ecrire une fonction `maxi : 'a list -> 'a option` qui prend en argument une liste et qui renvoie `None` si la liste est vide, et le maximum de la liste sinon.
       

In [3]:
let rec maxi lst =
   match lst with
   | [] -> None
   | t::q -> match maxi q with
             | None -> Some t
             | Some m -> Some (max m t)
;;

val maxi : 'a list -> 'a option = <fun>


<span style='color:Teal '>
    
En fait, si `'a` est un type ordonné, le type `'a option` hérite de l'ordre induit par celui sur le type `'a`, avec `None` strictement inférieur à tous les éléments. On pourrait donc écrire une des deux fonctions suivantes :
</span>

In [4]:
let rec maxi lst =
   match lst with
   | [] -> None
   | [x] -> Some x
   | t::q -> max (maxi q) (Some t)
;;

val maxi : 'a list -> 'a option = <fun>


In [5]:
let rec maxi lst =
   match lst with
   | [] -> None
   | t::q -> max (maxi q) (Some t)
;;

val maxi : 'a list -> 'a option = <fun>


Montrer que votre fonction termine.

<span style='color:Teal '>
Par récurrence sur la longueur $n$ de la liste :
    
- Si $n = 0$, la liste est vide et la fonction termine
- Si $n \geqslant 1$, supposons que la fonction termine pour une liste de longueur $n-1$ ;
    - ou bien n = 1 et la fonction termine ;
    - ou bien $n \geqslant 2$, alors le troisième cas s'applique et `q` est de longueur $n-1$, donc `maxi q` termine. Par conséquent, la fonction termine.

Finalement, la fonction termine pour toutes les listes.
</span>

# Exercice 2
    
1. On considère le type `signe` défini de la manière suivante :

In [6]:
type signe =
  | Plus
  | Moins
;;

type signe = Plus | Moins


Définir une fonction `changement_signe` de prenant en entrée un signe `s` et retournant le signe opposé à `s`. Préciser son type.

In [7]:
let changement_signe s =
  match s with
  | Plus -> Moins
  | Moins -> Plus
;;

val changement_signe : signe -> signe = <fun>


2. Après avoir défini une type `limite` comportant quatre constructeurs dont 3 paramétrés, on peut écrire la fonction `oppose : limite -> limite` ci-dessous, qui renvoie la valeur de la limite de l'opposée d'une suite réelle de limite `lim`.
```OCaml
let oppose lim =
    match lim with
    | FormeIndeterminee -> FormeIndeterminee
    | Infini signe -> Infini (changement_signe signe)
    | Zero signe -> Zero (changement_signe signe)
    | Reel valeur -> Reel (-. valeur)
;;
```

   - Définir le type `limite`.
   

In [8]:
type limite =
   | FormeIndeterminee
   | Infini of signe
   | Zero of signe
   | Reel of float
;;

type limite =
    FormeIndeterminee
  | Infini of signe
  | Zero of signe
  | Reel of float


In [9]:
let oppose lim =
 match lim with
 | FormeIndeterminee -> FormeIndeterminee
 | Infini signe -> Infini (changement_signe signe)
 | Zero signe -> Zero (changement_signe signe)
 | Reel valeur -> Reel (-. valeur)
;;

val oppose : limite -> limite = <fun>


   - Que valent `oppose (Zero Plus)` et `oppose (Reel 2022.)` ?

In [10]:
oppose (Zero Plus)

- : limite = Zero Moins


In [11]:
oppose (Reel 2022.)

- : limite = Reel (-2022.)


2. Définir une fonction `inverse` prenant en entrée une limite `lim` et renvoyant la limite de l'inverse d'une suite de limite `lim`. Préciser son type.

In [12]:
let inverse lim =
   match lim with
   | Infini s -> Zero s
   | Zero s -> Infini s
   | Reel x -> if x = 0. then FormeIndeterminee else Reel (1. /. x)
   | _ -> FormeIndeterminee
;;

val inverse : limite -> limite = <fun>


3. Définir une fonction `somme` prenant en entrée deux limites `lim1` et `lim2` renvoyant la limite de la somme de deux suites de limites respectives `lim1` et `lim2`.

  Dans le cas d'une somme de limite nulle, on prendra garde à distinguer les cas pour lesquels on connaît le signe de la somme au voisinage de $+\infty$.
  

In [13]:
let somme lim1 lim2 =
    match lim1, lim2 with
    | FormeIndeterminee, _ | _, FormeIndeterminee -> FormeIndeterminee
    | Infini s1, Infini s2 -> if s1 = s2 then Infini s1 else FormeIndeterminee
    | Infini s, _ | _, Infini s -> Infini s
    | Zero s1, Zero s2 -> if s1 = s2 then Zero s1 else Reel 0.
    | Zero _, Reel a | Reel a, Zero _ -> Reel a
    | Reel a, Reel b -> Reel (a +. b)
;;

val somme : limite -> limite -> limite = <fun>


4. Définir une fonction `produit` prenant en entrée deux limites `lim1` et `lim2` renvoyant la limite du produit de deux suites de limites respectives `lim1` et `lim2`.

  On pourra commencer par écrire une fonction prenant en entrée deux signes et appliquant la règle des signes pour le produit.

  Dans le cas où la limite du produit est nulle, on s'efforcera d'utiliser le constructeur `Zero` dès lors qu'il est possible de connaître le signe du produit au voisinage de $+\infty$.

In [14]:
let produit lim1 lim2 =
    match lim1, lim2 with
    | FormeIndeterminee, _ | _, FormeIndeterminee -> FormeIndeterminee
    | Infini s1, Infini s2 -> if s1 = s2 then Infini Plus else Infini Moins
    | Infini _, Zero _ | Zero _, Infini _ -> FormeIndeterminee
    | Infini s, Reel 0. | Reel 0., Infini s -> FormeIndeterminee
    | Infini s, Reel a | Reel a, Infini s -> Infini (if a > 0. then s else changement_signe s)
    | Zero s1, Zero s2 -> if s1 = s2 then Zero Plus else Zero Moins
    | Zero s, Reel 0. | Reel 0., Zero s -> Reel 0.
    | Zero s, Reel a | Reel a, Zero s -> Zero (if a > 0. then s else changement_signe s)
    | Reel a, Reel b -> Reel (a *. b)
;;

val produit : limite -> limite -> limite = <fun>


# Exercice

Les nombres de Fibonacci sont définis par :
    $$F_0 = 0, \quad F_1=1, \qquad \forall n \geqslant 0, F_{n+2}=F_{n+1}+F_n $$
    
**Notation** : $k$ et $m$ étant des nombres entiers, $k \gg m$ signifie que $k \geqslant m+2$.
    
On admet que tout entier $n$ a **une unique** décomposition :
    $$n = F_{k_1}+F_{k_2}+\dots + F_{k_r} \text{ où }k_1 \gg k_2 \gg \dots \gg k_r \gg 0$$
et que pour tout entier naturel $k$,
    $$F_{k+1} = F_k+F_{k-2}+F_{k-4}+\dots + F_{k-2i} + \dots + 1 \text{ pour tous les indices tels que }k-2i\geqslant 2$$
    
En utilisant cette propriété, on peut déduire assez facilement la décomposition de $n+1$ à partir de celle de $n$.

En effet, soit $n=F_{k_1}+F_{k_2}+\dots + F_{k_r}$ et soit $j \in ⟦ 1,r ⟧$ le plus petit entier tel que $\forall s, \quad s \in ⟦ j, r ⟧ \Rightarrow k_{s+1} = k_s-2$. Autrement dit, dans la décomposition de $n$ les indices diminuent de deux en deux, vers la droite, seulement à partir de $k_j$ :
    $$n=F_{k_1}+\dots+F_{k_{j-1}}+\left( F_{k_j}+F_{k_j-2}+F_{k_j-4}+\dots + F_{k_r}\right)$$
On distingue trois cas :
- Si $k_r>3$, alors $n+1 = F_{k_1}+F_{k_2}+\dots+F_{k_r}+F_2$ 
-  Si $k_r \leqslant 3$ et $j=1$, alors $n+1 = F_{k_1+1}$
-  Si $k_r \leqslant 3$ et $j>1$ , alors $n+1 =F_{k_1}+F_{k_2}+\dots+ F_{k_{j-1}}+F_{k_j+1}$

Ainsi, sachant que $1=F_2$, on déduit :
$1=F_2$ (cas 2) $\rightarrow \, 2=F_3$ (cas 2) $\rightarrow \, 3=F_4$ (cas 1) $\rightarrow \, 4=F_4+F_2$ (cas 2) $\rightarrow \, 5=F_5$ (cas 1) $\rightarrow \, 6=F_5+F_2$ (cas 3) $\rightarrow \, 7=F_5+F_3$ (cas 2) $\rightarrow \, 8=F_6$ (cas 1) $\rightarrow \dots$ 

Dans ce qui suit on représentera la décomposition d'un entier $n$ par le couple `(j,lst)` où `lst` est la liste ordonnée (par ordre décroissant) des indices des nombres de Fibonacci qui interviennent dans la décomposition de $n$ et `j` est l'entier qui indique le rang à partir duquel les éléments de `lst` diminuent de deux en deux.


*Exemple :*
- $n=33=F_8+F_6+F_4+F_2$ sera représenté par le couple `(1,[8;6;4;2])`.
- $n=40=F_9+F_5+F_2$ sera représenté par le couple `(3,[9;5;2])`.

1. Ecriture de quelques fonctions qui peuvent s'avérer utiles pour la suite :
- Écrire la fonction `ieme : int -> 'a list -> 'a` qui renvoie le `i`{ème} élément d'une liste `lst` ; `ieme 1 lst` renverra le premier élément de la liste `lst`.

In [15]:
let rec ieme i lst =
  match lst with
  | [] -> failwith "Longueur insuffisante"
  | t::q -> if i = 1
            then t
            else ieme (i-1) q
;;

val ieme : int -> 'a list -> 'a = <fun>


- Écrire la fonction `tete : int -> 'a list -> 'a list` qui renvoie une nouvelle liste construire en recopiant les `i` premiers éléments de la liste `lst`. Si cette sous-liste n'existe pas, la fonction renvoie la liste vide.

In [16]:
let rec tete i lst =
  match lst, i with
  | [], _ -> []
  | _, 0 -> []
  | t::_, 1->  [t]
  | t::q, _ -> let tq = tete (i-1) q in
               if tq = [] then [] else t::tq
;;

val tete : int -> 'a list -> 'a list = <fun>


- Écrire la fonction `queue : int -> 'a list -> 'a list` qui renvoie une nouvelle liste construite en supprimant dans la liste `lst` ses `i` premiers éléments. Si cette sous-liste n'existe pas, la fonction renvoie la liste vide.

In [17]:
let rec queue i lst =
  match lst, i with
  | [], _ -> []
  | _, 0 ->  lst
  | t::q, _ -> queue (i-1) q
;;

val queue : int -> 'a list -> 'a list = <fun>


- Ecrire la fonction `ajoute : 'a -> 'a list -> 'a list` qui prend en argument un élément `x` et une liste `lst` et qui renvoie la liste obtenue en ajoutant l'élément `x` à la fin de la liste `lst`.

In [18]:
let rec ajoute x lst =
  match lst with
  | [] -> [x]
  | t::q -> t::(ajoute x q)
;;

val ajoute : 'a -> 'a list -> 'a list = <fun>


- Ecrire la fonction `fusion : 'a list -> 'a list -> 'a list` qui prend en argument deux listes supposées triées dans l'ordre \textbf{décroissant} et qui renvoie une liste triée dans l'ordre décroissant et contenant les éléments des deux listes.

In [19]:
let rec fusion l1 l2 =
    match l1,l2 with
    | [],_ -> l2
    | _, [] -> l1
    | t1::q1, t2::q2 -> if t1 > t2
                        then t1::(fusion q1 l2)
                        else t2::(fusion l1 q2)
;;

val fusion : 'a list -> 'a list -> 'a list = <fun>


- Montrer que cette fonction termine.


<span style='color:Teal '>
Par induction sur le couple $(n_1,n_2)$ des longueurs respectives des listes `l1` et `l2` :
    
- Si $(n_1, n_2) = (0, 0)$, les deux listes sont vides donc la fonction termine
- Sinon, supposons que la fonction termine pour tous les couples d'entiers naturels strictement inférieurs à $(n_1,n_2)$ ;
    - si $n_1 = 0$ ou $n_2 = 0$, une des deux listes est vide et la fonction termine ;
    - sinon, on effectue un appel récursif pour deux listes de longueurs $(n_1-1,n_2)$ ou $(n_1, n_2-1)$, qui sont strictement plus petits que $(n_1, n_2)$, donc la fonction termine.

Finalement, la fonction termine pour toutes les listes.
    
*Remarque : il est bien évidemment possible de raisonner par récurrence sur la somme des deux longueurs*
</span>

    
On pourra de plus utiliser les fonctions `List.length`, `List.hd`, `List.tl`.

*Exemples :*
```OCaml
# ieme 3 [10; 8; 6; 2];;
 - : int = 6
# tete 3 [10; 8; 6; 2];;
 - : int list = [10; 8; 6]
# queue 3 [10; 8; 6; 2];;
 - : int list = [2]
# ajoute 1 [10; 8; 6; 2];;
 - : int list = [10; 8; 6; 2; 1]
```
*Dans la suite, il peut être pertinent de faire précéder la fonction demandée par d'autres définitions de fonctions utiles.*

2. Ecrire la fonction `enumere : int -> (int*int list) list` qui à partir de la décomposition $1=F_2$, engendre les décompositions en somme de Fibonacci de tous les entiers jusqu'à `n`. Le résultat de cette fonction doit être une liste de couples :

   `[(1,[2]);(1;[3]);(1;[4]);(1;[4;2]);(1;[5]);(2;[5;2]);(1;[5;3]);(1;[6]);...]`

<span style='color:Teal '>
    
 La fonction `calcule_j` prend en argument une liste $[k_1;\dots;k_r]$ telle que ${k_1 \gg \dots \gg k_r \gg 0}$ et renvoie $j \in [ 1, r ]$ tel que $\forall s \in [j, r], k_{s+1} = k_s-2$.
</span>

In [20]:
let rec calcule_j lst =
  match lst with
  | [] -> 0
  | [_] -> 1
  | t1::q -> let j2 = calcule_j q in
         if t1 > List.hd q + 2
         then j2 + 1
         else
           if j2 = 1
           then 1
           else j2 + 1
;;

val calcule_j : int list -> int = <fun>


<span style='color:Teal '>
    
 La fonction `suivant` prend en argument la liste représentant un entier $n$ et renvoie la liste représentant l'entier $n+1$.</span>

In [21]:
let suivant (j, lst) =
  let lgr = List.length lst in
  let kr = ieme lgr lst in
  if kr > 3
  then let lst_suivant = ajoute 2 lst in
       let new_j = (if kr = 4 then j else List.length lst_suivant)
       in (new_j, lst_suivant)
  else
    if j = 1
    then (1, [List.hd lst + 1])
    else
      let lst_suivant = ajoute (ieme j lst + 1) (tete (j-1) lst) in
      (calcule_j lst_suivant, lst_suivant)
;;

val suivant : int * int list -> int * int list = <fun>


<span style='color:Teal '>
    
 Enfin, la fonction locale `aux` prend en argument une liste `last` représentant un entier $p$, une liste `acc` des décompositions des entiers de $1$ à $p$ et le nombre d'éléments `nb` qu'il reste à calculer et renvoie la liste complétée avec les décompositions des `nb` entiers suivants.</span>

In [22]:
let enumere n =
  let rec aux last acc nb =
    match nb with
    | 0 -> acc
    | _ -> let next = suivant last in
           aux next (ajoute next acc) (nb-1)
  in
  let un = (1, [2]) in
  aux un [un] (n-1) 
;;

val enumere : int -> (int * int list) list = <fun>


<span style='color:Teal '>
    
*Remarque : l'utilisation de la fonction `ajoute` n'est pas idéale pour la complexité. Il pourrait être pertinent de construire la liste à l'envers, puis de la renverser.*
    </span>

3. Soit $n=F_{i_1}+F_{i_2}+\dots+F_{i_s}$ où $i_1  \geqslant i_2 \geqslant \dots \geqslant i_s \geqslant 2$ une somme quelconque de nombres de Fibonacci.

   Ecrire la fonction `decomposition : int list -> int list` qui renvoie la liste des indices des nombres de Fibonacci de l'unique décomposition de ${n=F_{k_1}+F_{k_2}+\dots + F_{k_r}}$ où $k_1 \gg k_2 \gg \dots \gg k_r \gg 0$.
   
    Si la fonction `decomposition` reçoit pour paramètre effectif la liste $[i_1;i_2;\dots;i_s]$ elle retourne pour résultat la liste $[k_1;k_2;\dots;k_r]$.

<span style='color:Teal '>
    
 Plutôt que de calculer l'entier correspondant à la somme puis sa décomposition, procédons par modifications successives de la liste $[i_1;\dots;i_s]$, en remarquant que :
- si $i_j=p$ et $i_{j-1} = p-1$, alors $F_{i_j} + F_{i_{j+1}} = F_{p+1}$ ;
- si $i_j=i_{j+1} = p$ avec $p \geqslant 4$, alors $F_{i_j} + F_{i_{j+1}} = F_{p+1}+F_{p-2}$ ;
- si $i_j=i_{j+1} = 3$, alors $F_{i_j} + F_{i_{j+1}} = F_{4}+F_{1} = F_{4}+F_{2}$ ;
- si $i_j=i_{j+1} = 2$,
    alors $F_{i_j} + F_{i_{j+1}} = 2 = F_{3}$ ;

On parcourt alors la liste jusqu'à trouver un indice $j$ tel que la condition $i_j \gg i_{j+1}$ n'est pas vérifiée, puis on applique la transformation correspondante (fonction `une_etape`). Pour cela, on utilise la fonction `insere` qui insère un élément à sa place dans une liste triée.

La fonction `une_etape` renvoie la liste obtenue après une éventuelle transformation, ainsi qu'un booléen indiquant si la liste a été modifiée.


Pour obtenir la décomposition attendue, on applique la fonction `une_etape` jusqu'à ce qu'il n'y ait plus de modification à apporter.
    </span>

In [23]:
let rec insere e lst =
  match lst with
  | [] -> [e]
  | t::q  -> if t > e
             then t::(insere e q)
             else e::lst
;;

let rec une_etape lst =
    match lst with
    | [] -> [], true
    | [_] -> lst, true
    | t1::t2::q ->
       match t1-t2 with
       | 0 ->
          begin
              match t1 with
              | 2 -> 3::q, false
              | 3 -> 4::(insere 2 q), false
              | _ -> (t1+1)::(insere (t1-2) q), false
            end
       | 1 -> (t1+1)::q, false
       | _ -> let qr, fini = une_etape (t2::q) in
              t1::qr, fini
;;


let rec decomposition liste =
  let lst, fini = une_etape liste in
  if fini
  then lst
  else decomposition lst
;;

val insere : 'a -> 'a list -> 'a list = <fun>


val une_etape : int list -> int list * bool = <fun>


val decomposition : int list -> int list = <fun>


<span style='color:Teal '>
 
*Remarque :  La terminaison de cette fonction ne semble pas assurée. On pourra vérifier que le triplet $\left(s, \sum_{j=1}^s i_j, \mathrm{Card}\, \{j \mid i_j = 3 \} \right)$ décroît strictement pour l'ordre lexicographique sur $\mathbb{N}^3$ à chaque appel de `decomposition`.*
    </span>

4. Ecrire la fonction `somme : int list -> int list -> int list` qui prend en arguments deux listes correspondant à l'unique décomposition de deux entiers et retourne pour résultat la liste correspondant à l'unique décomposition de la somme de ces deux entiers.

In [24]:
let somme a b = decomposition (fusion a b)
;;

val somme : int list -> int list -> int list = <fun>
